In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
#from category_encoders import OneHotEncoder
import warnings
import random
import os
plt.style.use('ggplot')
warnings.filterwarnings(action='ignore')
#import catboost

In [3]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.6 MB/s eta 0:00:00


In [4]:
from catboost import *

In [85]:
model = CatBoostRegressor(random_state=44, verbose=500, iterations = 5000, task_type ='GPU', learning_rate=0.03)

In [76]:
train = pd.read_csv('/content/drive/MyDrive/lg/train.csv')
test = pd.read_csv('/content/drive/MyDrive/lg/test.csv')

In [64]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [66]:
train = train.fillna(-1)
train = test.fillna(-1)

In [67]:
collist = train.describe().columns
arr=[]
for col in collist :
    adfuller_test = adfuller(train[[col]], autolag= "AIC")
    if adfuller_test[1] >=0.05 :
        arr.append(col)
train = train.drop(columns=arr)
test = test.drop(columns=arr)

In [77]:
train = train.drop(columns=arr)
test = test.drop(columns=arr)

In [68]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [78]:
col_list = train.columns
nan_list = []
nan_cnt = []
nan_col = []
full_list = []
for col in col_list:
    if train[col].isnull().sum() == 0 :
        full_list.append(col)
        continue
    nan_list.append([col, train[col].isnull().sum()])
    nan_cnt.append(train[col].isnull().sum())
    nan_col.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_list :
    if col[1] == 598 :
        del_col.append(col[0])
train = train.drop(columns=del_col)
test = test.drop(columns=del_col)
train.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN


In [79]:
trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

In [80]:
col_list = train.columns
nan_listA_31 = []
nan_cntA_31 = []
nan_colA_31 = []
full_listA_31 = []
for col in col_list:
    if trainA_31[col].isnull().sum() == 0 :
        full_listA_31.append(col)
        continue
    nan_listA_31.append([col, trainA_31[col].isnull().sum()])
    nan_cntA_31.append(trainA_31[col].isnull().sum())
    nan_colA_31.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listA_31 :
    if col[1] == len(trainA_31) :
        del_col.append(col[0])
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = trainA_31.columns
for col in col_list[6:] :
    if trainA_31[col].nunique()==1 :
        del_col.append(col)
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

In [81]:
col_list = train.columns
nan_listO = []
nan_cntO = []
nan_colO = []
full_listO = []
for col in col_list:
    if train_O_31[col].isnull().sum() == 0 :
        full_listO.append(col)
        continue
    nan_listO.append([col, train_O_31[col].isnull().sum()])
    nan_cntO.append(train_O_31[col].isnull().sum())
    nan_colO.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listO :
    if col[1] == len(train_O_31) :
        del_col.append(col[0])
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_O_31.columns
for col in col_list[6:] :
    if train_O_31[col].nunique()==1 :
        del_col.append(col)
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

In [82]:
col_list = train.columns
nan_listT = []
nan_cntT = []
nan_colT = []
full_listT = []
for col in col_list:
    if train_T_31[col].isnull().sum() == 0 :
        full_listT.append(col)
        continue
    nan_listT.append([col, train_T_31[col].isnull().sum()])
    nan_cntT.append(train_T_31[col].isnull().sum())
    nan_colT.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listT :
    if col[1] == len(train_T_31) :
        del_col.append(col[0])
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_T_31.columns
for col in col_list[6:] :
    if train_T_31[col].nunique()==1 :
        del_col.append(col)
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

In [83]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
testA_31_x = testA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_T_31_x = train_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','PRODUCT_CODE'])
test_T_31_x = test_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_O_31_x = train_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
test_O_31_x = test_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])

# classification
trainA_31_y_c = trainA_31['Y_Class']
train_T_31_y_c = train_T_31['Y_Class']
train_O_31_y_c = train_O_31['Y_Class']

# regression
trainA_31_y_r = trainA_31['Y_Quality']
train_T_31_y_r = train_T_31['Y_Quality']
train_O_31_y_r = train_O_31['Y_Quality']

In [84]:
print(trainA_31_x.shape,'\n',testA_31_x.shape,'\n',train_T_31_x.shape,'\n',test_T_31_x.shape,'\n',train_O_31_x.shape,'\n',test_O_31_x.shape)

(249, 1866) 
 (67, 1866) 
 (343, 549) 
 (239, 549) 
 (6, 499) 
 (4, 499)


In [86]:
trainA_31_x=trainA_31_x.fillna(-100)
testA_31_x=testA_31_x.fillna(-100)
train_T_31_x=train_T_31_x.fillna(-100)
test_T_31_x=test_T_31_x.fillna(-100)
train_O_31_x=train_O_31_x.fillna(-100)
test_O_31_x=test_O_31_x.fillna(-100)

In [87]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(trainA_31_x[i])
    trainA_31_x[i] = le.transform(trainA_31_x[i])
    
    for label in np.unique(testA_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    testA_31_x[i] = le.transform(testA_31_x[i]) 
print('Done.')

Done.


In [88]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T_31_x[i])
    train_T_31_x[i] = le.transform(train_T_31_x[i])
    
    for label in np.unique(test_T_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T_31_x[i] = le.transform(test_T_31_x[i]) 
print('Done.')

Done.


In [89]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_O_31_x[i])
    train_O_31_x[i] = le.transform(train_O_31_x[i])
    
    for label in np.unique(test_O_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_O_31_x[i] = le.transform(test_O_31_x[i]) 
print('Done.')

Done.


In [90]:
model.fit(trainA_31_x, trainA_31_y_r)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0098408	total: 74.9ms	remaining: 6m 14s
500:	learn: 0.0026371	total: 20.8s	remaining: 3m 6s
1000:	learn: 0.0019066	total: 45.9s	remaining: 3m 3s
1500:	learn: 0.0014172	total: 1m 8s	remaining: 2m 39s
2000:	learn: 0.0011401	total: 1m 29s	remaining: 2m 13s
2500:	learn: 0.0009680	total: 1m 51s	remaining: 1m 51s
3000:	learn: 0.0008445	total: 2m 13s	remaining: 1m 28s
3500:	learn: 0.0007569	total: 2m 33s	remaining: 1m 5s
4000:	learn: 0.0007022	total: 2m 55s	remaining: 43.9s
4500:	learn: 0.0006563	total: 3m 16s	remaining: 21.7s
4999:	learn: 0.0006179	total: 3m 38s	remaining: 0us


In [91]:
model.fit(train_T_31_x, train_T_31_y_r)
pred_t = model.predict(test_T_31_x)

0:	learn: 0.0047262	total: 10.9ms	remaining: 54.4s
500:	learn: 0.0016846	total: 11s	remaining: 1m 39s
1000:	learn: 0.0012530	total: 22.7s	remaining: 1m 30s
1500:	learn: 0.0009971	total: 32.5s	remaining: 1m 15s
2000:	learn: 0.0008137	total: 43.6s	remaining: 1m 5s
2500:	learn: 0.0006999	total: 55.4s	remaining: 55.3s
3000:	learn: 0.0006113	total: 1m 7s	remaining: 44.6s
3500:	learn: 0.0005245	total: 1m 18s	remaining: 33.5s
4000:	learn: 0.0004630	total: 1m 29s	remaining: 22.3s
4500:	learn: 0.0004090	total: 1m 39s	remaining: 11s
4999:	learn: 0.0003683	total: 1m 50s	remaining: 0us


In [92]:
model.fit(train_O_31_x, train_O_31_y_r)
pred_o = model.predict(test_O_31_x)

0:	learn: 0.0031761	total: 6.21ms	remaining: 31.1s
500:	learn: 0.0005046	total: 3.34s	remaining: 30s
1000:	learn: 0.0002630	total: 6.63s	remaining: 26.5s
1500:	learn: 0.0001312	total: 8.66s	remaining: 20.2s
2000:	learn: 0.0000667	total: 10.7s	remaining: 16.1s
2500:	learn: 0.0000465	total: 12.6s	remaining: 12.6s
3000:	learn: 0.0000190	total: 14.5s	remaining: 9.67s
3500:	learn: 0.0000077	total: 17.3s	remaining: 7.43s
4000:	learn: 0.0000040	total: 20.6s	remaining: 5.15s
4500:	learn: 0.0000030	total: 22.7s	remaining: 2.52s
4999:	learn: 0.0000028	total: 24.8s	remaining: 0us


In [93]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

In [94]:
testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

In [95]:
testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

test_T_31.loc[(test_T_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

test_O_31.loc[(test_O_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

In [96]:
submita = pd.read_csv('/content/drive/MyDrive/lg/sample_submission.csv')
submitt = pd.read_csv('/content/drive/MyDrive/lg/sample_submission.csv')
submito = pd.read_csv('/content/drive/MyDrive/lg/sample_submission.csv')

In [97]:
submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

In [98]:
pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('/content/drive/MyDrive/lg/statm.csv',index=False)

In [99]:
sub = pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID')

In [100]:
len(sub[sub['Y_Class']==0])

27

In [101]:
len(sub[sub['Y_Class']==2])

12